In [14]:
import argparse
import sys

def args_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument('--lr',         type=float, default=0.1,       help='learning rate')
    parser.add_argument('--optimizer',  type=str,   default='AdamW',    help='optimizer type')
    parser.add_argument('--iteration',  type=int,   default=250,        help='number of iterations')
    parser.add_argument('--gamma',      type=float, default=0.1,        help='gamma value')
    args, unknown = parser.parse_known_args()
    return args

args = args_parser()

In [13]:
import sys

sys.path.append("./datasets")
import os
import numpy as np
import pickle as pkl
from datasets.data_preprocess import data_preprocess
import logistic_regression
import datetime
from plot import plot_logreg
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
current_work_dir = os.getcwd()


def elapsed_time(func):
    def wrapper(*args, **kwargs):
        start_time = datetime.datetime.now()
        start_time.strftime("%H:%M:%S")
        result = func(*args, **kwargs)
        end_time = datetime.datetime.now()
        print("===" * 15)
        print("Elapsed time: ", end_time - start_time)
        print("===" * 15)
        return result

    return wrapper


weight_diff_list = []
obj_diff_list = []


@elapsed_time
def main_run():
    if __name__ == "__main__":
        try:
            (x_train, y_train), (x_test, y_test) = data_preprocess(args)

            print("learning rate: ", args.lr)
            print("Optimizer: ", args.optimizer)
            print("-------------------------")

            Model = logistic_regression.LogisticRegression(
                args=args, X_train=x_train, Y_train=y_train, X_test=x_test
            )

            weight_diff, obj_diff = Model.diff_cal(Model.weights)
            print("\n------------ Initial ------------")
            print("weight error: {:.4e}".format(weight_diff))
            print("objective error: {:.4e}".format(obj_diff))

            Eigvals = np.linalg.eigvals(Model.pre_Hessian)
            print("\nmax eigenvalue of Hessian:{:.4f}".format(np.max(Eigvals)))
            print("min eigenvalue of Hessian:{:.4f}".format(np.min(Eigvals)))

            for i in range(args.iteration):
                weight_diff, obj_diff = Model.update()
                print("\n------------ Iteration {} ------------".format(i + 1))
                print("weight error: {:.4e}".format(weight_diff))
                print("objective error: {:.4e}".format(obj_diff))
                weight_diff_list.append(weight_diff)
                obj_diff_list.append(obj_diff)

                if weight_diff / np.sqrt(Model.dimension) <= 1e-5:
                    break

            val = Model.getTest() > 0.5
            val2 = y_test > 0.5
            percent_correct = np.mean(val == val2) * 100
            print("Accuracy: {:.1f}%".format(percent_correct))

        except Exception as e:
            logger.error(
                f"Line {e.__traceback__.tb_lineno} | An error occurred: {e} "
            )


main_run()

file_name = "./results/{}_{}.pkl".format("logreg", args.optimizer)
file_name = os.path.join(current_work_dir, file_name)
with open(file_name, "wb") as f:
    pkl.dump([weight_diff_list, obj_diff_list], f)
plot_logreg()

-------------------------
training dataset:  (12665, 784)
testing dataset:   (2115, 784)
learning rate:  0.1
Optimizer:  AdamW
-------------------------
============= CVX solving =============
============= CVX solved =============

------------ Initial ------------
weight error: 1.0509e+00
objective error: 5.9037e-01

max eigenvalue of Hessian:9.8267
min eigenvalue of Hessian:0.1000
using fused adam: False

------------ Iteration 1 ------------
weight error: 2.1122e+00
objective error: 4.7528e-01
using fused adam: False

------------ Iteration 2 ------------
weight error: 2.0173e+00
objective error: 3.1843e-01
using fused adam: False

------------ Iteration 3 ------------
weight error: 2.4506e+00
objective error: 4.5069e-01
using fused adam: False

------------ Iteration 4 ------------
weight error: 2.8079e+00
objective error: 5.7819e-01
using fused adam: False

------------ Iteration 5 ------------
weight error: 2.8127e+00
objective error: 5.9767e-01
using fused adam: False

--------

usage: ipykernel_launcher.py [-h] [--lr LR] [--optimizer OPTIMIZER]
                             [--iteration ITERATION] [--gamma GAMMA]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/zephyrus/.local/share/jupyter/runtime/kernel-ea60a138-5f1b-4856-b264-133bf8549b85.json


SystemExit: 2